# Capstone 2: Updated Data Wrangling<a id='Data_Wrangling'></a>

## Contents<a id='Contents'></a>
* [Data Wrangling](#Data_Wrangling)
  * [Contents](#Contents)
  * [Introduction](#Introduction)
  * [Imports](#Imports)
  * [Load the Data](#Load_the_Data)
  * [Data Cleaning](#Data_Cleaning)
  * [Data Merging](#Data_Merging)
  * [Data Export](#Data_Export)

## Introduction<a id='Introduction'></a>

After initial data wrangling and exploratory data analysis, I determined that the initial features dataset had extra data for which there was no weekly sales data. I decided to double check my first attempt at data wrangling and found some areas of potential improvement and also setting aside data for future predictions.

## Imports<a id='Imports'></a>

In [303]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.optimize import curve_fit
import datetime as dt
import os
from library.sb_utils import save_file

## Load the Data<a id='Load_the_Data'></a>

In [304]:
features = pd.read_csv('C:/Users/jmhat/Downloads/Features.csv')
stores = pd.read_csv('C:/Users/jmhat/Downloads/stores.csv')
sales = pd.read_csv('C:/Users/jmhat/Downloads/sales.csv')

In [305]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [306]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


## Data Cleaning<a id='Data_Cleaning'></a>

Change the IsHoliday column to zeroes and ones.

In [307]:
features['IsHoliday'] = features['IsHoliday'].astype(int)

In [308]:
features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,0
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,1
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,0
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,0
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,28/06/2013,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,0
8186,45,05/07/2013,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,0
8187,45,12/07/2013,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,0
8188,45,19/07/2013,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,0


After an initial exploration of the data in a CSV editor, I found that the CPI data followed a polynomial trend. I then found the index values for each CPI range where the data was NaN. I found a stackoverflow example showing how to use curve fitting and applied that to my dataset.

In [309]:
# Function to curve fit to the data
def func(x, a, b, c, d, e, f):
    return a * (x ** 5) + b * (x ** 4) + c * (x ** 3) + d * (x ** 2) + e * x + f

# Initial parameter guess
guess = (0.5, 0.5, 0.5, 0.5, 0.5, 0.5)

CPI_new =  pd.DataFrame()

CPI_dict = {0:181, 182:363, 364:545, 546:727, 728:909, 910:1091, 1092:1273, 1274:1455, 1456:1637, 1638:1819, 1820:2001,
           2002:2183, 2184:2365, 2366:2547, 2548:2729, 2730:2911, 2912:3093, 3094:3275, 3276:3457, 3458:3639, 3640:3821,
           3822:4003, 4004:4185, 4186:4367, 4368:4549, 4550:4731, 4732:4913, 4914:5095, 5096:5277, 5278:5459, 5460:5641,
           5642:5823, 5824:6005, 6006:6187, 6188:6369, 6370:6551, 6552:6733, 6734:6915, 6916:7097, 7098:7279, 7280:7461,
           7462:7643, 7644:7825, 7826:8007, 8008:8189}

for k, v in CPI_dict.items():
    
    df = features.loc[k:v, 'CPI'].to_frame()
    #df = part.to_frame()

    # Create copy of data to remove NaNs for curve fitting
    fit_df = df.dropna()

    # Place to store function parameters for each column
    col_params = {}

    # Curve fit each column
    for col in fit_df.columns:
        # Get x & y
        x = fit_df.index.astype(float).values
        y = fit_df[col].values
        # Curve fit column and get curve parameters
        params = curve_fit(func, x, y, guess)
        # Store optimized parameters
        col_params[col] = params[0]

    # Extrapolate each column
    for col in df.columns:
        # Get the index values for NaNs in the column
        x = df[pd.isnull(df[col])].index.astype(float).values
        # Extrapolate those points with the fitted function
        df[col][x] = func(x, *col_params[col])
    
    CPI_new = CPI_new.append(df, ignore_index=True)


<ipython-input-309-9eb43dc524fe>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CPI_new = CPI_new.append(df, ignore_index=True)
<ipython-input-309-9eb43dc524fe>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CPI_new = CPI_new.append(df, ignore_index=True)
<ipython-input-309-9eb43dc524fe>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CPI_new = CPI_new.append(df, ignore_index=True)
<ipython-input-309-9eb43dc524fe>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CPI_new = CPI_new.append(df, ignore_index=True)
<ipython-input-309-9eb43dc524fe>:44: FutureWarning: The frame.append method is deprecated and will be removed from panda

In [310]:
CPI_new

,CPI
0,211.096358
1,211.242170
2,211.289143
3,211.319643
4,211.350143
...,...
8185,190.676143
8186,190.462385
8187,190.237661
8188,190.001749


The CPI data was updated in the orginal dataset through the iteration code above. I'll now look at the unemployment data. I initially found that the unemployment values, although continuous, do not update more than every three or so months. To fill in the missing values, since we don't know where these stores are located, I will use the forward fill method since not many values are missing.

In [311]:
features['Unemployment'] = features['Unemployment'].fillna(method='ffill')

In [312]:
features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,0
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,1
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,0
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,0
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,28/06/2013,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,190.676143,8.335,0
8186,45,05/07/2013,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,190.462385,8.335,0
8187,45,12/07/2013,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,190.237661,8.335,0
8188,45,19/07/2013,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,190.001749,8.335,0


In [313]:
features.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI                0
Unemployment       0
IsHoliday          0
dtype: int64

The missing values are solely markdowns that either weren't tracked or there were no markdowns. For those missing values, I will simply fill them with zeros.

In [314]:
features = features.fillna(0)

In [315]:
features.isna().sum()

Store           0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
dtype: int64

In [316]:
features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,0
1,1,12/02/2010,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,1
2,1,19/02/2010,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,0
3,1,26/02/2010,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,0
4,1,05/03/2010,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,28/06/2013,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,190.676143,8.335,0
8186,45,05/07/2013,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,190.462385,8.335,0
8187,45,12/07/2013,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,190.237661,8.335,0
8188,45,19/07/2013,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,190.001749,8.335,0


Below, I've taken a look at the date column, but I will update the date on all the dataframes later on.

In [317]:
len(features['Date'].unique())

182

In [318]:
features['Date'].value_counts(ascending=True)

05/02/2010    45
27/04/2012    45
04/05/2012    45
11/05/2012    45
18/05/2012    45
              ..
22/04/2011    45
29/04/2011    45
06/05/2011    45
20/05/2011    45
26/07/2013    45
Name: Date, Length: 182, dtype: int64

Now, I'll inspect the stores dataframe.

In [319]:
stores

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875
5,6,A,202505
6,7,B,70713
7,8,A,155078
8,9,B,125833
9,10,B,126512


In [320]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [321]:
stores.isna().sum()

Store    0
Type     0
Size     0
dtype: int64

In [322]:
sales

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False
...,...,...,...,...,...
421565,45,98,28/09/2012,508.37,False
421566,45,98,05/10/2012,628.10,False
421567,45,98,12/10/2012,1061.02,False
421568,45,98,19/10/2012,760.01,False


In [323]:
sales.isna().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [324]:
len(sales['Date'].unique())

143

## Data Merging<a id='Data_Merging'></a>

Here, I want to group all departments per store and sum the weekly sales of each department per date per store.

In [325]:
new_group = sales.groupby(by=['Store', 'Date'])['Weekly_Sales'].sum()

In [326]:
New_sales = new_group.to_frame().reset_index()

In [327]:
New_sales

,Store,Date,Weekly_Sales
0,1,01/04/2011,1495064.75
1,1,01/06/2012,1624477.58
2,1,01/07/2011,1488538.09
3,1,01/10/2010,1453329.50
4,1,02/03/2012,1688420.76
...,...,...,...
6430,45,30/07/2010,716859.27
6431,45,30/09/2011,698986.34
6432,45,30/12/2011,869403.63
6433,45,31/08/2012,734297.87


In [328]:
sales['Date'].value_counts()

23/12/2011    3027
25/11/2011    3021
16/12/2011    3013
09/12/2011    3010
17/02/2012    3007
              ... 
09/07/2010    2903
16/07/2010    2901
20/08/2010    2901
27/08/2010    2898
13/08/2010    2896
Name: Date, Length: 143, dtype: int64

At this point, I'll circle back to the features dataframe and split the dates with sales data from the dates with no sales data. This will allow me to predict future sales in the modelling section.

In [329]:
Feat =  pd.DataFrame()
Feat_dict = {0:142, 182:324, 364:506, 546:688, 728:870, 910:1052, 1092:1234, 1274:1416, 1456:1598, 1638:1780, 1820:1962,
           2002:2144, 2184:2326, 2366:2508, 2548:2690, 2730:2872, 2912:3054, 3094:3236, 3276:3418, 3458:3600, 3640:3782,
           3822:3964, 4004:4146, 4186:4328, 4368:4510, 4550:4692, 4732:4874, 4914:5056, 5096:5238, 5278:5420, 5460:5602,
           5642:5784, 5824:5966, 6006:6148, 6188:6330, 6370:6512, 6552:6694, 6734:6876, 6916:7058, 7098:7240, 7280:7422,
           7462:7604, 7644:7786, 7826:7968, 8008:8150}

for k, v in Feat_dict.items():
    Feat = Feat.append(features.loc[k:v], ignore_index=True)

<ipython-input-329-816fbe19b911>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat = Feat.append(features.loc[k:v], ignore_index=True)


In [330]:
Feat

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,0
1,1,12/02/2010,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,1
2,1,19/02/2010,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,0
3,1,26/02/2010,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,0
4,1,05/03/2010,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6430,45,28/09/2012,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,0
6431,45,05/10/2012,64.89,3.985,5046.74,0.00,18.82,2253.43,2340.01,192.170412,8.667,0
6432,45,12/10/2012,54.47,4.000,1956.28,0.00,7.89,599.32,3990.54,192.327265,8.667,0
6433,45,19/10/2012,56.47,3.969,2004.02,0.00,3.18,437.73,1537.49,192.330854,8.667,0


Here, I'll change the dates to ordinal values for better use in machine learning.

In [331]:
Feat['Date'] = pd.to_datetime(Feat['Date'], dayfirst=True).dt.date

In [332]:
Feat['Date'] = Feat['Date'].map(dt.datetime.toordinal)

In [333]:
Feat

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,733808,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,0
1,1,733815,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,1
2,1,733822,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,0
3,1,733829,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,0
4,1,733836,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6430,45,734774,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,0
6431,45,734781,64.89,3.985,5046.74,0.00,18.82,2253.43,2340.01,192.170412,8.667,0
6432,45,734788,54.47,4.000,1956.28,0.00,7.89,599.32,3990.54,192.327265,8.667,0
6433,45,734795,56.47,3.969,2004.02,0.00,3.18,437.73,1537.49,192.330854,8.667,0


I'll also change the dates to ordinal values in the sales dataset.

In [334]:
New_sales['Date'] = pd.to_datetime(New_sales['Date'], dayfirst=True).dt.date

In [335]:
New_sales['Date'] = New_sales['Date'].map(dt.datetime.toordinal)

In [336]:
New_sales

,Store,Date,Weekly_Sales
0,1,734228,1495064.75
1,1,734655,1624477.58
2,1,734319,1488538.09
3,1,734046,1453329.50
4,1,734564,1688420.76
...,...,...,...
6430,45,733983,716859.27
6431,45,734410,698986.34
6432,45,734501,869403.63
6433,45,734746,734297.87


I'll now merge the separate features dataset and sales dataset together for analysis.

In [337]:
merge_1 = pd.merge(Feat, New_sales, how='left', left_on=['Store', 'Date'], right_on=['Store', 'Date'])

In [338]:
merge_1

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Weekly_Sales
0,1,733808,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,0,1643690.90
1,1,733815,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,1,1641957.44
2,1,733822,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,0,1611968.17
3,1,733829,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,0,1409727.59
4,1,733836,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,0,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,45,734774,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,0,713173.95
6431,45,734781,64.89,3.985,5046.74,0.00,18.82,2253.43,2340.01,192.170412,8.667,0,733455.07
6432,45,734788,54.47,4.000,1956.28,0.00,7.89,599.32,3990.54,192.327265,8.667,0,734464.36
6433,45,734795,56.47,3.969,2004.02,0.00,3.18,437.73,1537.49,192.330854,8.667,0,718125.53


Before I merge the stores dataset to the total dataset, I want to change the store type column to a dummy variable for machine learning.

In [339]:
stores = pd.get_dummies(stores, columns=['Type'], drop_first=True)

In [340]:
stores

,Store,Size,Type_B,Type_C
0,1,151315,0,0
1,2,202307,0,0
2,3,37392,1,0
3,4,205863,0,0
4,5,34875,1,0
5,6,202505,0,0
6,7,70713,1,0
7,8,155078,0,0
8,9,125833,1,0
9,10,126512,1,0


I'll now make a new dataset with all the dates for just the data with sales values.

In [341]:
retail_data = pd.merge(merge_1, stores, how='left', left_on=['Store'], right_on=['Store'])

In [342]:
retail_data

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Weekly_Sales,Size,Type_B,Type_C
0,1,733808,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,0,1643690.90,151315,0,0
1,1,733815,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,1,1641957.44,151315,0,0
2,1,733822,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,0,1611968.17,151315,0,0
3,1,733829,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,0,1409727.59,151315,0,0
4,1,733836,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,0,1554806.68,151315,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,45,734774,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,0,713173.95,118221,1,0
6431,45,734781,64.89,3.985,5046.74,0.00,18.82,2253.43,2340.01,192.170412,8.667,0,733455.07,118221,1,0
6432,45,734788,54.47,4.000,1956.28,0.00,7.89,599.32,3990.54,192.327265,8.667,0,734464.36,118221,1,0
6433,45,734795,56.47,3.969,2004.02,0.00,3.18,437.73,1537.49,192.330854,8.667,0,718125.53,118221,1,0


I now need to separate the data with no sales information to use for prediction analysis later.

In [343]:
Feat_nosales =  pd.DataFrame()
Feat_nosales_dict = {143:181, 325:363, 507:545, 689:727, 871:909, 1053:1091, 1235:1273, 1417:1455, 1599:1637, 1781:1819, 1963:2001,
                     2145:2183, 2327:2365, 2509:2547, 2691:2729, 2873:2911, 3055:3093, 3237:3275, 3419:3457, 3601:3639, 3783:3821,
                     3965:4003, 4147:4185, 4329:4367, 4511:4549, 4693:4731, 4875:4913, 5057:5095, 5239:5277, 5421:5459, 5603:5641,
                     5785:5823, 5967:6005, 6149:6187, 6331:6369, 6513:6551, 6695:6733, 6877:6915, 7059:7097, 7241:7279, 7423:7461,
                     7605:7643, 7787:7825, 7969:8007, 8151:8189}

for k, v in Feat_nosales_dict.items():
    Feat_nosales = Feat_nosales.append(features.loc[k:v], ignore_index=True)

<ipython-input-343-bc3a9aae1a2c>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_nosales = Feat_nosales.append(features.loc[k:v], ignore_index=True)


In [344]:
Feat_nosales

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,02/11/2012,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,0
1,1,09/11/2012,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,0
2,1,16/11/2012,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,0
3,1,23/11/2012,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,1
4,1,30/11/2012,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1750,45,28/06/2013,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,190.676143,8.335,0
1751,45,05/07/2013,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,190.462385,8.335,0
1752,45,12/07/2013,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,190.237661,8.335,0
1753,45,19/07/2013,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,190.001749,8.335,0


I'll also need to change the date data to ordinal values.

In [345]:
Feat_nosales['Date'] = pd.to_datetime(Feat_nosales['Date'], dayfirst=True).dt.date

In [346]:
Feat_nosales['Date'] = Feat_nosales['Date'].map(dt.datetime.toordinal)

In [347]:
Feat_nosales

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,734809,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,0
1,1,734816,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,0
2,1,734823,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,0
3,1,734830,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,1
4,1,734837,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1750,45,735047,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,190.676143,8.335,0
1751,45,735054,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,190.462385,8.335,0
1752,45,735061,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,190.237661,8.335,0
1753,45,735068,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,190.001749,8.335,0


In [348]:
retail_data_nosales = pd.merge(Feat_nosales, stores, how='left', left_on=['Store'], right_on=['Store'])

In [349]:
retail_data_nosales

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Size,Type_B,Type_C
0,1,734809,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,0,151315,0,0
1,1,734816,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,0,151315,0,0
2,1,734823,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,0,151315,0,0
3,1,734830,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,1,151315,0,0
4,1,734837,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573,0,151315,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750,45,735047,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,190.676143,8.335,0,118221,1,0
1751,45,735054,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,190.462385,8.335,0,118221,1,0
1752,45,735061,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,190.237661,8.335,0,118221,1,0
1753,45,735068,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,190.001749,8.335,0,118221,1,0


## Data Export<a id='Data_Export'></a>

Finally, I'll export the datasets for use in the machine learning notebook.

In [296]:
# save the data to a new csv file
datapath = '../data'
save_file(retail_data, 'retail_data.csv', datapath)

Writing file.  "../data\retail_data.csv"


In [297]:
# save the data to a new csv file
datapath = '../data'
save_file(retail_data_nosales, 'retail_data_nosales.csv', datapath)

Writing file.  "../data\retail_data_nosales.csv"
